In [ ]:
import json
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

from utils.evaluation import compute_SMAPE, evaluate_return_predictions

In [ ]:
trades = pd.read_csv("../results/trades.csv", sep=';')
statistics = json.load(open("../results/trades_statistics.json", "r"))
len(trades), len(statistics)

In [ ]:
trades

In [ ]:
statistics

In [ ]:
def compute_trading_performance(df: pd.DataFrame) -> pd.DataFrame:
    performance = {}
    for model in df["MODEL"].unique():
        df_model = df[df["MODEL"] == model]
        buy_price = df_model["BUY_PRICE"].values
        sell_price_gt = df_model["SELL_PRICE"].values
        sell_price_pr = df_model["PREDICTED_PRICE"].values
        win_gt = sell_price_gt > buy_price
        win_pr = sell_price_pr > buy_price
        metrics = evaluate_return_predictions(win_gt, win_pr)
        performance[model] = {
            "RETURN_MAE": round(np.mean(df_model["RETURN_AE"]), 4),
            "PRICE_SMAPE": round(compute_SMAPE(sell_price_gt, sell_price_pr), 2),
            "ACCURACY": accuracy_score(win_gt, win_pr),
            "PRECISION": metrics["Precision"],
            "RECALL": metrics["Recall"],
            "F1": metrics["F1"],
        }
    performance = pd.DataFrame(performance).transpose()
    performance = performance.reset_index(names="Model")
    return performance